In [6]:
# Import required packages 
import tensorflow as tf
import image_modeling   # import image_modeling.py file
import tensorflow_hub as hub
import datetime
import csv
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
#Load csv-data
image_dir = "../images/"
train_data = pd.read_csv('../data/train.csv')
train_data.image_id = train_data.image_id.apply(lambda x: x.strip()+".JPG")
test_data = pd.read_csv('../data/test.csv')
test_data.image_id = test_data.image_id.apply(lambda x: x.strip()+".JPG")

extra_data = pd.read_csv('../data/extra_images_loc.csv')
extra_data.image_id = extra_data.image_id.apply(lambda x: x.strip()+".JPG")

#Get unique_turtle_ids from train.csv
unique_turtle_ids = list(extra_data['turtle_id'].unique())
#Add category for new turtle for test set
unique_turtle_ids.append("new_turtle")
#Get number of images for train/test split
split = 0.75
lines = round(len(extra_data)*split)
length_data = len(extra_data)

#We set some parameters for the model
HEIGHT = 224 #image height
WIDTH = 224 #image width
CHANNELS = 3 #image RGB channels
CLASS_NAMES = list(extra_data['image_location'].unique())
NCLASSES = len(CLASS_NAMES)
BATCH_SIZE = 32
SHUFFLE_BUFFER = 10 * BATCH_SIZE
AUTOTUNE = tf.data.experimental.AUTOTUNE

TRAINING_SIZE = lines
VALIDATION_SIZE = length_data - lines                    
VALIDATION_STEPS = VALIDATION_SIZE // BATCH_SIZE

In [8]:
def preprocess(augment = True):
    if augment == True:
        train_datagen = ImageDataGenerator(
                rotation_range     = 40,
                width_shift_range  = 0.2,
                height_shift_range = 0.2,
                rescale            = 1./255,
                shear_range        = 0.2,
                zoom_range         = 0.2,
                horizontal_flip    = True,
                fill_mode          = 'nearest')

        test_datagen = ImageDataGenerator(rescale=1./255)
    
    else:
        train_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen  = ImageDataGenerator(rescale=1./255)
        
    return train_datagen, test_datagen

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
#x_col value : which will be the name of column(in dataframe) having file names
#y_col value : which will be the name of column(in dataframe) having class/label

def use_image_generator(train_datagen, test_datagen, training=True): 
    
    if training == True:
        # Augment and scale images for training
        train_generator = train_datagen.flow_from_dataframe(dataframe =extra_data[0:lines], 
                directory   = image_dir,
                x_col       = "image_id" ,
                y_col       = "image_location",
                target_size = (HEIGHT, WIDTH),
                batch_size  = BATCH_SIZE,
                classes     = CLASS_NAMES,
                class_mode  = 'categorical',
                shuffle     = True)
                #save_to_dir="output/",  if you wanna save the cropped images
                #save_prefix="",
                #save_format='png')

        # Scale images for validation
        validation_generator = test_datagen.flow_from_dataframe(dataframe = extra_data[lines:], 
                directory    = image_dir,
                x_col        = "image_id",
                y_col        = "image_location",
                target_size  = (HEIGHT, WIDTH),
                batch_size   = BATCH_SIZE,
                classes      = CLASS_NAMES,
                class_mode   = 'categorical',
                shuffle      = True)
        
        return train_generator, validation_generator
    
    else:
        # Scale images for testing, no target provided and returned
        test_generator = test_datagen.flow_from_dataframe(dataframe = test_data, 
                directory   = image_dir,
                x_col       = "image_id",
                target_size = (HEIGHT, WIDTH),
                batch_size  = BATCH_SIZE,
                class_mode  = None,
                shuffle     = False)
            
        return test_generator

In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [11]:
#change the last layer
for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final softmax layer with 3 nodes for classification output
x = layers.Dense(NCLASSES, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
train_datagen, test_datagen = preprocess()
train_generator, validation_generator = use_image_generator(train_datagen, test_datagen, training=True)
    
inception =  model.fit(
        train_generator, 
        validation_data=validation_generator,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=30,
        callbacks=[tensorboard_callback])

Found 7994 validated image filenames belonging to 3 classes.
Found 2664 validated image filenames belonging to 3 classes.
Epoch 1/30
249/249 [==============================] - 211s 836ms/step - loss: 1.2151 - accuracy: 0.5805 - val_loss: 0.4319 - val_accuracy: 0.6426
Epoch 2/30
249/249 [==============================] - 216s 866ms/step - loss: 0.4387 - accuracy: 0.6091 - val_loss: 0.3651 - val_accuracy: 0.6505
Epoch 3/30
249/249 [==============================] - 223s 894ms/step - loss: 0.4330 - accuracy: 0.6074 - val_loss: 0.3619 - val_accuracy: 0.6520
Epoch 4/30
249/249 [==============================] - 1283s 5s/step - loss: 0.4158 - accuracy: 0.6166 - val_loss: 0.3698 - val_accuracy: 0.6614
Epoch 5/30
249/249 [==============================] - 548s 2s/step - loss: 0.4175 - accuracy: 0.6188 - val_loss: 0.3763 - val_accuracy: 0.6502
Epoch 6/30
249/249 [==============================] - 220s 882ms/step - loss: 0.4210 - accuracy: 0.6119 - val_loss: 0.3812 - val_accuracy: 0.6532
Epoch 7

In [12]:
model.save('extra_images_location')

2022-02-11 01:23:16.256450: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: extra_images_location/assets
